In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u422-b05-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.3
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 1.8.0_422
Branch HEAD
Compiled by user haejoon.lee on 2024-09-09T05:20:05Z
Revision 32232e9ed33bb16b93ad58cfde8b82e0f07c0970
Url https://github.com/apache/spark
Type --help for more information.


**transaction dataset**



In [5]:
from google.colab import files
files.upload()

Saving test.csv to test (3).csv


{'test (3).csv': b'txnid,txndate,custid,amount,product,category,city,state,spendby\r\n0,06-26-2011,4000001,40.33,Exercise & Fitness,Cardio Machine Accessories,Clarksville,Tennessee,credit\r\n1,05-26-2011,4000002,198.44,Exercise & Fitness,Weightlifting Gloves,Long Beach,California,credit\r\n2,06-01-2011,4000002,5.58,Exercise & Fitness,Weightlifting Machine Accessories,Anaheim,California,credit\r\n3,06-05-2011,4000003,198.19,Gymnastics,Gymnastics Rings,Milwaukee,Wisconsin,credit\r\n4,12-17-2011,4000002,98.81,Team Sports,Field Hockey,Nashville  ,Tennessee,credit\r\n5,02-14-2011,4000004,193.63,Outdoor Recreation,Camping & Backpacking & Hiking,Chicago,Illinois,credit\r\n6,10-28-2011,4000005,27.89,Puzzles,Jigsaw Puzzles,Charleston,South Carolina,credit\r\n7,07-14-2011,4000006,96.01,Outdoor Play Equipment,Sandboxes,Columbus,Ohio,credit\r\n8,01-17-2011,4000006,10.44,Winter Sports,Snowmobiling,Des Moines,Iowa,credit\r\n9,05-17-2011,4000006,152.46,Jumping,Bungee Jumping,St. Petersburg,Florida,cr

***Fetching CSV Data ***

In [6]:
data = spark.read.csv("test.csv", header=True)
data.show()

+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-------+
|txnid|   txndate| custid|amount|             product|            category|          city|         state|spendby|
+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-------+
|    0|06-26-2011|4000001| 40.33|  Exercise & Fitness|Cardio Machine Ac...|   Clarksville|     Tennessee| credit|
|    1|05-26-2011|4000002|198.44|  Exercise & Fitness|Weightlifting Gloves|    Long Beach|    California| credit|
|    2|06-01-2011|4000002|  5.58|  Exercise & Fitness|Weightlifting Mac...|       Anaheim|    California| credit|
|    3|06-05-2011|4000003|198.19|          Gymnastics|    Gymnastics Rings|     Milwaukee|     Wisconsin| credit|
|    4|12-17-2011|4000002| 98.81|         Team Sports|        Field Hockey|   Nashville  |     Tennessee| credit|
|    5|02-14-2011|4000004|193.63|  Outdoor Recreation|Camping & Backpac...|       Chicag

**1. Find the total number of transaction for each customer id**

In [7]:
from pyspark.sql.functions import *
data.groupBy('custid').count().show()

+-------+-----+
| custid|count|
+-------+-----+
|4000002|    6|
|4000008|   10|
|4000001|    8|
|4000003|    3|
|4000005|    5|
|4000009|    6|
|4000010|    6|
|4000004|    5|
|4000007|    6|
|4000006|    5|
+-------+-----+



**2. Find the total amount of total transaction amount of each customer **

In [8]:
data.withColumn('amount',col('amount').cast('float')).groupBy('custid').agg(format_number(sum('amount'), 2).alias('Total Customer Transaction Value')).show()

+-------+--------------------------------+
| custid|Total Customer Transaction Value|
+-------+--------------------------------+
|4000002|                          706.97|
|4000008|                          859.42|
|4000001|                          651.05|
|4000003|                          527.59|
|4000005|                          325.15|
|4000009|                          457.83|
|4000010|                          447.09|
|4000004|                          337.06|
|4000007|                          699.55|
|4000006|                          539.38|
+-------+--------------------------------+



**3. Find the total number of customer who have spent by credit card**

In [11]:
data.filter(data.spendby == 'credit').groupBy('custid').count().show()
print("Total Number of customers puchased by credit ")
data.select('custid').filter(data.spendby == 'credit').distinct().count()

+-------+-----+
| custid|count|
+-------+-----+
|4000002|    5|
|4000008|   10|
|4000001|    7|
|4000003|    3|
|4000005|    4|
|4000009|    6|
|4000010|    6|
|4000004|    1|
|4000007|    6|
|4000006|    5|
+-------+-----+

Total Number of customers puchased by credit 


10

**4. Find the total number of customer who have spent by cash**

In [12]:
data.filter(data.spendby == 'cash').groupBy('custid').count().show()
print("Total Number of customers puchased by cash : ")
data.select('custid').filter(data.spendby == 'cash').distinct().count()

+-------+-----+
| custid|count|
+-------+-----+
|4000002|    1|
|4000001|    1|
|4000005|    1|
|4000004|    4|
+-------+-----+

Total Number of customers puchased by cash : 


4